In [ ]:
!pip install -U datasets evaluate transformers langchain langchain-openai faiss-cpu langchain-community llamaapi


from dotenv import load_dotenv
from openai import OpenAI
from transformers import pipeline
from datasets import load_dataset, Dataset
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from collections import defaultdict
from google.colab import drive
import re
import csv
import json
import os
import torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: pyarrow
    Found existing

In [ ]:
HF_TOKEN =#insertyourkey


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load dataset
dataset = load_dataset(
    "json",
    data_files="/content/drive/MyDrive/role_aware_squad.json",
    split="train"
)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
#skip first thousand coz we already used them for training and evaluating
def balance_role_dataset(dataset, per_role_count=500):

    role_data = defaultdict(lambda: {"answerable": [], "unanswerable": []})

    for i, example in enumerate(dataset):
        if i < 1000:
          continue
        for role in ["EMPLOYEE", "CUSTOMER"]:
            answerable = example["role_answerable"][role]
            entry = {
                "question": example["question"],
                "original_context": example["original_context"],
                "original_answer": example["original_answer"],
                "role_context": example["role_contexts"][role],
                "is_answerable": answerable,
                "role": role
            }
            category = "answerable" if answerable else "unanswerable"
            role_data[role][category].append(entry)

    # Sample balanced set
    balanced_examples = []
    for role in role_data:
        ans = role_data[role]["answerable"]
        unans = role_data[role]["unanswerable"]

        sampled = ans[:per_role_count] + unans[:per_role_count]
        balanced_examples.extend(sampled)

    return balanced_examples

In [ ]:
balanced_dataset = balance_role_dataset(dataset)

In [ ]:
def final_prompt_employee(query, context):
    return f"""

    You are an assistant who answers questions for an EMPLOYEE of the company.
    The EMPLOYEE is not allowed to see any PERSON or GPE Entities.
    You must only answer based on the text provided below.
    Provide the most accurate and concise answer possible.
    If an answer contains a PERSON or GPE Entity respond by saying "The information is not provided in the source."

    [BEGIN SOURCE TEXT]
    {context}
    [END SOURCE TEXT]

    Please answer the question
    Question: {query}
    """


def final_prompt_customer(query, context):
    return f"""

    You are an assistant who answers questions for an CUSTOMER of a company.
    The CUSTOMER is not allowed to see any PERSON, ORG, or GPE Entities.
    You must only answer based on the text provided below.
    Provide the most accurate and concise answer possible.
    If an answer contains a PERSON, ORG, or GPE Entity respond by saying "The information is not provided in the source."

    [BEGIN SOURCE TEXT]
    {context}
    [END SOURCE TEXT]

    Please answer the question
    Question: {query}
    """

In [ ]:
def is_actual_answer(response):

    response_clean = response.strip().lower()

    # List of phrases that indicate no answer was found
    denial_phrases = [
        "is not provided",
        "not provided",
        "is not available",
        "not provided in the source",
        "not mentioned in the source",
        "the source does not provide",
        "cannot be found in the source",
    ]


    # Check if response matches any denial phrase
    for phrase in denial_phrases:
        if phrase in response_clean:
            return False

    return True

In [ ]:
# store results
output_path = '/content/drive/MyDrive/llm_rag_eval_heuristic_results_prompt_engineering_1.csv'
# columns
fieldnames = [
    "index", "role", "question", "context",
    "llm_answer", "llm_says_answerable",
    "original_answer", "is_heuristically_answerable"
]

# check if file exists
file_exists = os.path.isfile(output_path)

# open file
with open(output_path, mode='a', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    if not file_exists:
        writer.writeheader()

In [ ]:
#process complete rag with prompt engineering
results = []
existing_indices = set()

if os.path.isfile(output_path):
    with open(output_path, mode='r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            existing_indices.add(int(row['index']))

index_counter = 0
for i in range(2000):
  index_counter += 1
  if index_counter in existing_indices:
    continue
  question = balanced_dataset[i]['question']
  context = balanced_dataset[i]['original_context']

  # Prompt + LLM
  if balanced_dataset[i]['role'] == 'CUSTOMER':
    prompt = final_prompt_customer(question, context)
  else:
    prompt = final_prompt_employee(question, context)


  client = OpenAI(
    base_url="https://gr8l7l65v3sk6xz2.us-east-1.aws.endpoints.huggingface.cloud/v1/",
    api_key=HF_TOKEN
  )

  r = client.chat.completions.create(
        model = "tgi",
        messages = [
          {
            "role": "user",
            "content": prompt
          }
        ],
        max_tokens=512
      )
  response = r.choices[0].message.content
  bool_answer = is_actual_answer(response)

  with open(output_path, mode='a', newline='', encoding='utf-8') as f:
          writer = csv.DictWriter(f, fieldnames=fieldnames)
          writer.writerow({
              "index": i,
              "role": balanced_dataset[i]['role'],
              "question": question,
              "context": context,
              "llm_answer": response,
              "llm_says_answerable": bool_answer,
              "original_answer": balanced_dataset[i]['original_answer'],
              "is_heuristically_answerable": balanced_dataset[i]['is_answerable']
          })



